# Core Concepts and Architectures in Ag2

## Architectures:

The multi-agent design of the architecture is easier to maintain, understand and extend than a single agent system. Even for the single agent based interface, a multi-agent implementation can potentially make the system more modular, and easier for developers to add or remove components of functionality. It's very important to recognize that the multi-agent architecture is a good way to build a single agent. While not obvious, it has a root in the society of mind theory by Marvin Minsky in 1986. Starting from simple agents, one can compose and coordinate them effectively to exhibit a higher level of intelligence.

## Orchestration Patterns
Patterns are the foundation of group chat orchestration - they define how agents take turns speaking and collaborating. Each pattern offers a different approach to determining which agent speaks next, giving you control over the conversation flow.



## Pattern Types

AG2 provides five orchestration patterns to structure agent interactions. Each pattern determines how agents take turns and collaborate, allowing you to tailor the conversation flow to your needs.

| Pattern            | Description                                                        | Best For                                               |
|--------------------|--------------------------------------------------------------------|--------------------------------------------------------|
| **DefaultPattern** | Relies solely on explicitly defined agent handoffs                 | Fine-grained control over workflow transitions         |
| **AutoPattern**    | Uses LLM to select the next speaker based on message content       | Dynamic conversations where the next speaker depends on context |
| **RoundRobinPattern** | Agents speak in a predetermined sequential order                | Workflows where each agent should contribute in turn    |
| **RandomPattern**  | Randomly selects the next agent (excluding the current speaker)    | Generating diverse perspectives or brainstorming        |
| **ManualPattern**  | Prompts the user to select the next speaker                        | Educational scenarios or debugging complex workflows    |

### DefaultPattern: Explicit Handoff Control

The **DefaultPattern** offers you complete, fine-grained control over the conversation flow by requiring explicitly defined handoffs between agents. With this pattern, you must specify exactly which agent should speak next at each step. If no handoff is defined, the conversation will end after the current agent's response.

**Key Characteristics:**

- **Explicit agent transitions:** You must define all agent handoffs manually.
- **No automatic speaker selection:** The system will not choose the next agent for you.
- **Conversation termination:** If a valid handoff is not specified, the conversation stops.
- **Maximum control:** Ideal for workflows where you need precise management of agent interactions.

In [ ]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import DefaultPattern
from autogen.agentchat.group import AgentTarget, RevertToUserTarget, OnCondition, StringLLMCondition

llm_config = LLMConfig(api_type="openai", model="gpt-5-mini")

with llm_config:
    triage_agent = ConversableAgent(
        name="triage_agent",
        system_message="""You are a triage agent. For each user query,
        identify whether it is a technical issue or a general question. Route
        technical issues to the tech agent and general questions to the general agent.
        Do not provide suggestions or answers, only route the query."""
    )

    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You solve technical problems like software bugs
        and hardware issues."""
    )

    general_agent = ConversableAgent(
        name="general_agent",
        system_message="You handle general, non-technical support questions."
    )

user = ConversableAgent(name="user", human_input_mode="ALWAYS")

pattern = DefaultPattern(
    initial_agent=triage_agent,
    agents=[triage_agent, tech_agent, general_agent],
    user_agent=user,
    group_manager_args={"llm_config": llm_config}
)

# Define explicit handoffs for each agent
triage_agent.handoffs.add_llm_conditions([
        OnCondition(
            target=AgentTarget(tech_agent),
            condition=StringLLMCondition(prompt="When the user query is related to technical issues."),
        ),
        OnCondition(
            target=AgentTarget(agent=general_agent),
            condition=StringLLMCondition(prompt="When the user query is related to general questions."),
        )
    ]
)

tech_agent.handoffs.set_after_work(RevertToUserTarget())
general_agent.handoffs.set_after_work(RevertToUserTarget())

result, context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=10
)

## AutoPattern: LLM-Powered Speaker Selection

The **AutoPattern** uses an LLM to intelligently select the next speaker based on the conversation context, creating a more dynamic and adaptive conversation flow.

### Key Characteristics

- **Automatically selects the next speaker** using an LLM
- **Analyzes conversation context** to determine the most appropriate agent
- Agents can still use **explicit handoffs** when needed
- Creates **natural, adaptive conversations**

In [ ]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

llm_config = LLMConfig(api_type="openai", model="gpt-5-mini")

with llm_config:
    triage_agent = ConversableAgent(
        name="triage_agent",
        system_message="""You are a triage agent. For each user query,
        identify whether it is a technical issue or a general question. Route
        technical issues to the tech agent and general questions to the general agent.
        Do not provide suggestions or answers, only route the query."""
    )

    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You solve technical problems like software bugs
        and hardware issues."""
    )

    general_agent = ConversableAgent(
        name="general_agent",
        system_message="You handle general, non-technical support questions."
    )

user = ConversableAgent(name="user", human_input_mode="ALWAYS")

pattern = AutoPattern(
    initial_agent=triage_agent,
    agents=[triage_agent, tech_agent, general_agent],
    user_agent=user,
    group_manager_args={"llm_config": llm_config}
)

result, context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=10
)

## RoundRobinPattern: Sequential Speaker Rotation

The `RoundRobinPattern` enforces a predictable, fixed rotation among agents, where each agent takes turns speaking in the exact order specified in the pattern configuration.

**Key Characteristics:**
- Agents speak in a fixed, sequential order.
- The order is determined by the list of agents provided to the pattern.
- Behavior is predictable and deterministic.


In [ ]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import RoundRobinPattern

llm_config = LLMConfig(api_type="openai", model="gpt-5-mini")

with llm_config:
    triage_agent = ConversableAgent(
        name="triage_agent",
        system_message="""You are a triage agent. For each user query,
        identify whether it is a technical issue or a general question. Route
        technical issues to the tech agent and general questions to the general agent.
        Do not provide suggestions or answers, only route the query."""
    )

    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You solve technical problems like software bugs
        and hardware issues."""
    )

    general_agent = ConversableAgent(
        name="general_agent",
        system_message="You handle general, non-technical support questions."
    )

user = ConversableAgent(name="user", human_input_mode="ALWAYS")

pattern = RoundRobinPattern(
    initial_agent=triage_agent,
    agents=[triage_agent, general_agent, tech_agent],
    user_agent=user,
    group_manager_args={"llm_config": llm_config}
)

result, context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=10
)

## RandomPattern: Non-deterministic Speaker Selection

The `RandomPattern` introduces unpredictability by randomly choosing the next speaker (excluding the current one). This approach is useful for generating a variety of perspectives within a conversation.

**Key Characteristics:**
- Randomly selects the next speaker (excluding the current speaker)
- Produces unpredictable, non-deterministic conversations
- Encourages diverse viewpoints and responses


In [ ]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import RandomPattern

llm_config = LLMConfig(api_type="openai", model="gpt-5-mini")

with llm_config:
    triage_agent = ConversableAgent(
        name="triage_agent",
        system_message="""You are a triage agent. For each user query,
        identify whether it is a technical issue or a general question. Route
        technical issues to the tech agent and general questions to the general agent.
        Do not provide suggestions or answers, only route the query."""
    )

    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You solve technical problems like software bugs
        and hardware issues."""
    )

    general_agent = ConversableAgent(
        name="general_agent",
        system_message="You handle general, non-technical support questions."
    )

user = ConversableAgent(name="user", human_input_mode="ALWAYS")

pattern = RandomPattern(
    initial_agent=triage_agent,
    agents=[triage_agent, tech_agent, general_agent],
    user_agent=user,
    group_manager_args={"llm_config": llm_config}
)

result, context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=10
)

## ManualPattern: Human-Directed Speaker Selectio
The `ManualPattern` gives you full control over the conversation flow by always returning to the user agent after each agent's turn. This allows you to manually choose which agent should speak next
**Key Characteristics:**
- Always reverts to the user agent after an agent speaks
- User manually selects the next agent to respond
- Provides maximum human oversight and control
- Ideal for debugging, demonstrations, or educational purposes


In [ ]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import ManualPattern

llm_config = LLMConfig(api_type="openai", model="gpt-5-mini")

with llm_config:
    triage_agent = ConversableAgent(
        name="triage_agent",
        system_message="""You are a triage agent. For each user query,
        identify whether it is a technical issue or a general question. Route
        technical issues to the tech agent and general questions to the general agent.
        Do not provide suggestions or answers, only route the query."""
    )

    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You solve technical problems like software bugs
        and hardware issues."""
    )

    general_agent = ConversableAgent(
        name="general_agent",
        system_message="You handle general, non-technical support questions."
    )

user = ConversableAgent(name="user", human_input_mode="ALWAYS")

pattern = ManualPattern(
    initial_agent=triage_agent,
    agents=[triage_agent, tech_agent, general_agent],
    user_agent=user,
    group_manager_args={"llm_config": llm_config}
)

result, context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=10
)